In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
227,2024-08-04,Mundo Jogos Olímpicos,06:00,Japão F,Bélgica F,3.51,1.30,144.5,1.88,1.90,6.5,2.02,1.17,UXRJ0uVe,0.284900,0.769231,0.531915,0.526316,0.054131,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,96.75,687.3,0.0,1.0,0.0,0.0,0.649774,0.007483,0.376828,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
228,2024-08-04,Indonésia Ibl,09:00,Pelita Jaya,Satria Muda,2.02,1.67,161.5,1.83,1.83,1.5,1.91,1.58,fTvYpwF8,0.495050,0.598802,0.546448,0.546448,0.093852,0.0,0.0,NaN,NaN,142.318,63.526017,0.446367,150.230,53.979873,0.359315,0.00,0.0,167.28,116.2,0.0,0.0,0.0,0.0,0.134139,0.000000,0.133722,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
229,2024-08-04,México Lnbp,21:00,Abejas,Soles,1.59,2.15,168.5,1.85,1.81,-3.5,1.88,2.08,CAjKVfUa,0.628931,0.465116,0.540541,0.552486,0.094047,0.6,0.0,NaN,NaN,163.968,32.610258,0.198882,NaN,NaN,NaN,135.46,0.0,127.20,226.0,0.0,0.0,0.0,0.0,0.211754,0.015456,0.071425,-2.25,-0.45,-1.311111,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
230,2024-08-04,México Lnbp,21:00,Mineros,Panteras,1.71,1.97,165.5,1.88,1.78,-2.5,1.93,2.03,KzfSTYbC,0.584795,0.507614,0.531915,0.561798,0.092410,0.0,0.0,NaN,NaN,NaN,NaN,NaN,178.206,45.180420,0.253529,0.00,0.0,0.00,202.5,0.0,0.0,0.0,0.0,0.099917,0.038640,0.035712,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,México Lnbp,Abejas,Soles,168.5,1.85,0.9999,Over
1,21:00,México Lnbp,Mineros,Panteras,165.5,1.88,1.0000,Over
